Define your variables (That's the only cells you need to change). Then run the whole notebook

In [1]:
method = 'SyNRA'
metric = 'mattes'
path_to_data = '/home/tli_project/Desktop/Data/T4/stage2/220321/220321_raw/'
save_path = '/home/tli_project/Desktop/Data/T4/stage2/220321/220321_analysis/'
group_by = '220321_brain3.2_LP40_P48.30'
ch_names = ['GFP', 'Red']
save_name = '220321_P48.30'
compile = True
filter = True

In [2]:
if save_path[-1] != '/':
    save_path += '/'

Load packages and define functions

In [3]:
import os
from timeit import default_timer as timer

In [4]:
import utils.datautils as datautils
import utils.registration as registration

/home/tli_project/.local/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


Running the pipeline

In [5]:
# reading list of 3D images into dictionary of channels as indicated in ch_names, and applying median filter on the last ch
image_4D = datautils.read_files(path_to_data, group_by ,compile=compile, ch_names=ch_names, filter=filter,
                                order=False, save=True, save_path=save_path, save_file=group_by+'_raw', 
                                xy_pixel=0.076, z_pixel=0.4)

first 5 files
['/home/tli_project/Desktop/Data/T4/stage2/220310/220310_raw/220310_brain2.1_LP40_P48_00001.tif', '/home/tli_project/Desktop/Data/T4/stage2/220310/220310_raw/220310_brain2.1_LP40_P48_00002.tif', '/home/tli_project/Desktop/Data/T4/stage2/220310/220310_raw/220310_brain2.1_LP40_P48_00003.tif', '/home/tli_project/Desktop/Data/T4/stage2/220310/220310_raw/220310_brain2.1_LP40_P48_00004.tif', '/home/tli_project/Desktop/Data/T4/stage2/220310/220310_raw/220310_brain2.1_LP40_P48_00005.tif']


compiling_files:  99%|█████████▉| 97/98 [43:07<00:26, 26.61s/it]<tifffile.TiffFile '220310_brain2.1_…0_P48_00098.tif'> asarray failed to reshape (172, 1024, 1024) to (8, 10, 2, 1024, 1024)


dict_keys(['GFP', 'Red']) <class 'list'> 98
compiling the GFP channel
compiling the Red channel


/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter '4D_GFP_220310_b…LP40_P48_raw.tif'> truncating ImageJ file

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter '4D_Red_220310_b…LP40_P48_raw.tif'> truncating ImageJ file



In [6]:
# measuring the cosine_similarity check of GFP channel before registration
pre_checks = datautils.similarity_4D(image_4D[ch_names[0]], 
                                    save=True, 
                                    save_path=save_path, 
                                    save_file='pre_checks.csv')

In [7]:
# running antspy, saving output into temp 3D files, and then compiling them to 4D and finally deleting the temp 3D files
# name_shifts = method+csv_name
registration.antspy_drift_corr(img_4D_g=image_4D[ch_names[0]],
                                img_4D_r=image_4D[ch_names[1]], 
                                ch_names=ch_names,
                                save_path=save_path, 
                                save_name=save_name,
                                # ref_t = 0,
                                ref_t = int(len(image_4D[ch_names[0]])/2),
                                drift_corr=method, metric=metric) 

applying_antspy: 100%|██████████| 90/90 [7:37:35<00:00, 305.07s/it]  


In [8]:
# compiling the registered 3D images into 4D and deleting the temp files for both channels 
gfp_name = ch_names[0]+'_'+method+'_4D_'+save_name+'.tif'
gfp_list = datautils.get_file_names(path=save_path, 
                                    group_by=ch_names[0]+'_'+method, 
                                    order=False, nested_files=False, criteria='tif')
gfp_4D = datautils.files_to_4D(files_list=gfp_list, ch_names=[ch_names[0]], filter=False, 
                                save=True, save_path=save_path, save_file=gfp_name,
                                xy_pixel=0.0764616, z_pixel=0.4)
for file in gfp_list:
    os.remove(file)

# measuring the cosine_similarity check of GFP channel after registration
post_checks = datautils.similarity_4D(gfp_4D[ch_names[0]], save=True, 
                                        save_path=save_path, 
                                        save_file=method+'_checks.csv')

del gfp_4D

red_name = ch_names[1]+'_'+method+'_4D_'+save_name+'.tif'
red_list = datautils.get_file_names(path=save_path, 
                                    group_by=ch_names[1]+'_'+method, 
                                    order=False, 
                                    nested_files=False, criteria='tif')
red_4D = datautils.files_to_4D(files_list=red_list, ch_names=[ch_names[1]], filter=False,
                                save=True, save_path=save_path, save_file=red_name,
                                xy_pixel=0.0764616, z_pixel=0.4)
for file in red_list:
    os.remove(file)

# deleting the final compiled 4D images
del red_4D 

first 5 files
['/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/GFP_SyNRA_220331_P48.45_001.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/GFP_SyNRA_220331_P48.45_002.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/GFP_SyNRA_220331_P48.45_003.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/GFP_SyNRA_220331_P48.45_004.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/GFP_SyNRA_220331_P48.45_005.tif']


dict_keys(['GFP']) <class 'list'> 89
compiling the GFP channel


/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter '4D_GFP_GFP_SyNR…20331_P48.45.tif'> truncating ImageJ file



first 5 files
['/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/Red_SyNRA_220331_P48.45_001.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/Red_SyNRA_220331_P48.45_002.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/Red_SyNRA_220331_P48.45_003.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/Red_SyNRA_220331_P48.45_004.tif', '/home/tli_project/Desktop/Data/T4/stage2/220331/220331_analysis/Red_SyNRA_220331_P48.45_005.tif']


dict_keys(['Red']) <class 'list'> 89
compiling the Red channel


/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter '4D_Red_Red_SyNR…20331_P48.45.tif'> truncating ImageJ file



In [9]:
registration.phase_corr_4D(image_4D, sigma=5, xy_pixel=1, 
                            z_pixel=1, ch_names=ch_names, 
                            ref_ch=ch_names[0],                      
                            save=True, save_path=save_path,
                            save_file=save_name, save_shifts=True)

applying phase_corr: 100%|██████████| 88/88 [1:28:25<00:00, 60.29s/it]
/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter 'PhaseCorr_GFP_220331_P48.45.tif'> truncating ImageJ file

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter 'PhaseCorr_Red_220331_P48.45.tif'> truncating ImageJ file



({'GFP': array([[[[  -1,    1,   -1, ...,    1,    3,   -1],
           [  -3,   -1,    0, ...,    2,   -2,   -3],
           [  -1,   -1,    0, ...,   -1,   -3,    3],
           ...,
           [   8,    0,   -4, ...,   -1,    1,   -5],
           [  -3,    1,   -2, ...,  174,   -6,    1],
           [  -1,    1,   99, ...,   -1,   -1,    1]],
  
          [[  -2,    0,   -2, ...,   -6,    6,    2],
           [   2,   -2,    2, ...,    2,   -1,   -3],
           [  -4,    1,   -1, ...,    6,    1,   -6],
           ...,
           [   0,    2,    3, ...,    0,   -1,   -2],
           [   1,    2,   -1, ...,    0,    0,   -1],
           [   1,    0,    1, ...,    2,    2,   -2]],
  
          [[   0,   -2,    1, ...,    0,   -2,   -2],
           [   0,    1,   -1, ...,   -6,    3,    4],
           [  -1,    2,   -1, ...,    4,    0,   -2],
           ...,
           [  50,    2,   -1, ...,   -2,    2,   -3],
           [  -4,   -1,   -2, ...,   -1,   -2,   -1],
           [  58,  

In [ ]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)

Error: Canceled future for execute_request message before replies were done